# Setup

In [1]:
!nvidia-smi

Sat Jan  7 17:40:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    30W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q transformers gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 97.3 MB/s eta 0:00:00


In [3]:
import transformers
import torch
from transformers import AutoTokenizer, AutoModel

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
torch.cuda.empty_cache()

In [4]:
tokenizer = AutoTokenizer.from_pretrained('Geotrend/bert-base-bg-cased')

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/659 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/93.8k [00:00<?, ?B/s]

In [5]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = AutoModel.from_pretrained('Geotrend/bert-base-bg-cased')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 3)
    
    def forward(self, ids, mask, token_type_ids):
        output_1 = self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids).pooler_output
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

### Download and load model

In [6]:
!gdown "1RamIB822hKPQ98xsAQLb1hOctqFvniqa&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=1RamIB822hKPQ98xsAQLb1hOctqFvniqa&confirm=t
To: /content/sentiment-bg-model
100% 382M/382M [00:02<00:00, 160MB/s]


In [8]:
model = torch.load('sentiment-bg-model')

# Inference

In [9]:
MAX_LEN = 200

In [37]:
prompt = 'Войната по пътищата е безмилостна! Двама души загинаха днес на автомагистрала Тракия.'
# prompt = 'Марс е планета от Слънчевата система.'
# prompt = 'Успех за България! Наши ученици спечелиха престижни награди на европейски конкурс за млади учени.'

In [38]:
inputs = tokenizer.encode_plus(prompt, None, add_special_tokens=True, max_length=MAX_LEN, pad_to_max_length=True, return_token_type_ids=True)

ids = torch.tensor(inputs['input_ids'], dtype=torch.long).unsqueeze(0).to(device)
mask = torch.tensor(inputs['attention_mask'], dtype=torch.long).unsqueeze(0).to(device)
token_type_ids = torch.tensor(inputs["token_type_ids"], dtype=torch.long).unsqueeze(0).to(device)

In [39]:
output = model(ids, mask, token_type_ids)
['NEGATIVE', 'NEUTRAL', 'POSITIVE'][torch.argmax(output)]

'NEGATIVE'